In [ ]:
import os
import sys
from pathlib import Path
from typing import List, Tuple, Dict

import IPython
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from lasso.dyna import Binout
from matplotlib.figure import Figure

notebook_path = Path(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"])
project_dir = notebook_path.parent.parent
sys.path.append(str(project_dir))
import src.utils.custom_log as custom_log
from src.utils.local_displacement import get_displ_along_axis
from src.utils.set_rcparams import set_rcparams

os.chdir(project_dir)
LOG = custom_log.init_logger(log_lvl=10, start_msg=f"Start in {Path().cwd()}")

In [45]:
sim_dir: Path = Path(r"Q:\Val_Chain_Sims\Honda_Accord_2014_Original_THOR_2_7\Assemblies_1")
#sim_dir: Path = Path(r"Q:\Val_Chain_Sims\Honda_Accord_2014_Sled_with_HIII_RuntimeMin\Assemblies_1")
cases: List[str] = ["Oblique_Left", "Oblique_Right", "Full_Frontal"]
#cases = [c + "_DR" for c in cases]

In [46]:
node_ids: Dict[str, int] = {
    "B_PillarSillLeft": 82000028,
    "B_PillarSillRight": 82000020,
    "RearSillDriver": 82000012,
    "RearSillPassenger": 82000004,
    "Head": 67100100,  # 67000001
    "Chest": 67700300,  # 67000736,
    "Pelvis": 67800100,  # 67002091,
}

In [47]:
def get_data(binout_dir:Path) -> Tuple[np.ndarray, np.ndarray]:
    """_summary_

    Args:
        d3_dir (Path): directory with binout files

    Returns:
        Tuple[np.ndarray, np.ndarray]: node ids with shape (n_nodes, 1) and node displacements with shape (n_nodes, m_timestamps, o_dimensions)
    """
    LOG.info("Reading binouts from %s", binout_dir)
    binout = Binout(filepath=str(binout_dir / "binout*"))

    ds = np.array([binout.read("nodout", f'{d}_coordinate') for d in "xyz"]).T
    nids = binout.read('nodout', "ids")

    LOG.info("Got node ids shape %s and node coordinates shape %s", nids.shape, ds.shape)

    return nids, ds

In [ ]:
data: Dict[str, Tuple[np.ndarray, np.ndarray]] = {c: get_data(binout_dir=sim_dir / c) for c in cases}

In [ ]:
LOG.info("Get local displacement driver side")
ds_x_displ_dr: Dict[str, np.ndarray] = {
    case: get_displ_along_axis(
        nodes_coord=data[case][1],
        root_coord=data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillLeft"])[0][0], :, :],
        direction_coord=data[case][1][np.nonzero(data[case][0] == node_ids["RearSillDriver"])[0][0], :, :],
        as_displacement=True,
    )
    for case in cases
}
LOG.info("Got local displacement driver side: %s", [(c, ds_x_displ_dr[c].shape) for c in cases])

LOG.info("Get local displacement passenger side")
ds_x_displ_pa: Dict[str, np.ndarray] = {
    case: get_displ_along_axis(
        nodes_coord=data[case][1],
        root_coord=data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillRight"])[0][0], :, :],
        direction_coord=data[case][1][np.nonzero(data[case][0] == node_ids["RearSillPassenger"])[0][0], :, :],
        as_displacement=True,
    )
    for case in cases
}
LOG.info("Got local displacement passenger side: %s", [(c, ds_x_displ_pa[c].shape) for c in cases])

In [ ]:
LOG.info("Get local displacement front side")
ds_y_displ_front: Dict[str, np.ndarray] = {
    case: get_displ_along_axis(
        nodes_coord=data[case][1],
        root_coord=data[case][1][np.nonzero(data[case][0] == node_ids["RearSillDriver"])[0][0], :, :],
        direction_coord=data[case][1][np.nonzero(data[case][0] == node_ids["RearSillPassenger"])[0][0], :, :],
        as_displacement=True,
    )
    for case in cases
}
LOG.info("Got local displacement front side: %s", [(c, ds_y_displ_front[c].shape) for c in cases])

LOG.info("Get local displacement rear side")
ds_y_displ_rear: Dict[str, np.ndarray] = {
    case: get_displ_along_axis(
        nodes_coord=data[case][1],
        root_coord=data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillLeft"])[0][0], :, :],
        direction_coord=data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillRight"])[0][0], :, :],
        as_displacement=True,
    )
    for case in cases
}
LOG.info("Got local displacement rear side: %s", [(c, ds_y_displ_rear[c].shape) for c in cases])

In [ ]:
ds_z_displ_dr: Dict[str, np.ndarray] = {}
ds_z_displ_pa: Dict[str, np.ndarray] = {}

for case in cases:
    LOG.info("Get local displacement driver side")

    root_coord = data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillLeft"])[0][0], :, :]
    z_axis = np.cross(
        data[case][1][np.nonzero(data[case][0] == node_ids["RearSillDriver"])[0][0], :, :]
        - data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillLeft"])[0][0], :, :],
        data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillRight"])[0][0], :, :]
        - data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillLeft"])[0][0], :, :],
    )

    z_axis /= np.linalg.norm(z_axis, axis=1)[:, np.newaxis]
    direction_coord = root_coord + z_axis

    ds_z_displ_dr[case] = get_displ_along_axis(
        nodes_coord=data[case][1],
        root_coord=root_coord,
        direction_coord=direction_coord,
        as_displacement=True,
    )

    LOG.info("Get local displacement passenger side")

    root_coord = data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillRight"])[0][0], :, :]
    z_axis = np.cross(
        data[case][1][np.nonzero(data[case][0] == node_ids["RearSillPassenger"])[0][0], :, :]
        - data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillRight"])[0][0], :, :],
        data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillLeft"])[0][0], :, :]
        - data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillRight"])[0][0], :, :],
    )

    z_axis /= np.linalg.norm(z_axis, axis=1)[:, np.newaxis]
    direction_coord = root_coord + z_axis

    ds_z_displ_pa[case] = get_displ_along_axis(
        nodes_coord=data[case][1],
        root_coord=root_coord,
        direction_coord=direction_coord,
        as_displacement=True,
    )


LOG.info("Got local displacement driver side: %s", [(c, ds_x_displ_dr[c].shape) for c in cases])
LOG.info("Got local displacement passenger side: %s", [(c, ds_x_displ_pa[c].shape) for c in cases])

In [ ]:
for c in cases:
    fig, ax = plt.subplots(ncols=5, figsize=(23, 5))
    fig.suptitle(c)

    for region in ["Head", "Chest", "Pelvis"]:
        # x
        x0 = Binout(filepath=str(sim_dir / c / "binout*")).as_df("nodout", "x_coordinate")[str(node_ids[region])].iloc[0]
        ax[0].plot(
            Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time"),
            x0 - ds_x_displ_dr[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            label=region + "Driver",
        )
        ax[0].plot(
            Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time"),
            x0 - ds_x_displ_pa[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            label=region + "Passenger",
        )

        # y
        y0 = Binout(filepath=str(sim_dir / c / "binout*")).as_df("nodout", "y_coordinate")[str(node_ids[region])].iloc[0]
        ax[1].plot(
            Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time"),
            y0 - ds_y_displ_front[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            label=region + "Front",
        )
        ax[1].plot(
            Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time"),
            y0 - ds_y_displ_rear[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            label=region + "Rear",
        )

        # z
        z0 = Binout(filepath=str(sim_dir / c / "binout*")).as_df("nodout", "z_coordinate")[str(node_ids[region])].iloc[0]
        ax[2].plot(
            Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time"),
            z0 + ds_z_displ_dr[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            label=region + "Driver",
        )
        ax[2].plot(
            Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time"),
            z0 + ds_z_displ_pa[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            label=region + "Passenger",
        )

        # xz
        ax[3].plot(
            x0 - ds_x_displ_dr[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            z0 + ds_z_displ_dr[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            label=region + "Driver",
        )
        ax[3].plot(
            x0 - ds_x_displ_pa[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            z0 + ds_z_displ_pa[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            label=region + "Passenger",
        )
        ax[3].arrow(
            x0,
            z0,
            -ds_x_displ_dr[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :][200],
            ds_z_displ_dr[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :][200],
            head_width=20,
        )

        # xy
        ax[4].plot(
            x0 - ds_x_displ_dr[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            y0 - ds_y_displ_front[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            label=region + "Driver",
        )
        ax[4].plot(
            x0 - ds_x_displ_pa[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            y0 - ds_y_displ_rear[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :],
            label=region + "Passenger",
        )
        ax[4].arrow(
            x0,
            y0,
            -ds_x_displ_dr[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :][200],
            -ds_y_displ_front[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :][200],
            head_width=20,
        )

    ax[0].grid()
    ax[0].legend()
    ax[0].set_xlabel("time [ms]")
    ax[0].set_ylabel("x displacement [mm]")

    ax[1].grid()
    ax[1].legend()
    ax[1].set_xlabel("time [ms]")
    ax[1].set_ylabel("y displacement [mm]")

    ax[2].grid()
    ax[2].legend()
    ax[2].set_xlabel("time [ms]")
    ax[2].set_ylabel("z displacement [mm]")

    ax[3].grid()
    ax[3].legend()
    ax[3].set_xlabel("x displacement [mm]")
    ax[3].set_ylabel("z displacement [mm]")

    ax[4].grid()
    ax[4].legend()
    ax[4].set_xlabel("x displacement [mm]")
    ax[4].set_ylabel("y displacement [mm]")

    fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
cs = ["r", "b", "g"]
for i, case in enumerate(cases):
    dst = (
        data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillLeft"])[0][0], :, :]
        - data[case][1][np.nonzero(data[case][0] == node_ids["RearSillDriver"])[0][0], :, :]
    )
    dst = np.linalg.norm(dst, axis=1)
    dst -= dst[0]
    ax.plot(Binout(filepath=str(sim_dir / case / "binout*")).read("nodout", "time"), dst, label=case + "DR", ls="-", c=cs[i])

    dst = (
        data[case][1][np.nonzero(data[case][0] == node_ids["B_PillarSillRight"])[0][0], :, :]
        - data[case][1][np.nonzero(data[case][0] == node_ids["RearSillPassenger"])[0][0], :, :]
    )
    dst = np.linalg.norm(dst, axis=1)
    dst -= dst[0]
    ax.plot(Binout(filepath=str(sim_dir / case / "binout*")).read("nodout", "time"), dst, label=case + "PA", ls="-.", c=cs[i])
ax.legend()
ax.grid()

In [ ]:
qq = pd.read_csv(r"C:\Users\plaschkies\Downloads\extracted.csv.zip", index_col=0)
qq

In [ ]:
region = "Head"
c = "Oblique_Left"

fig, ax = plt.subplots()
ax.plot(qq.index, qq[f"01{region.upper()}LOC0TH50DSXD"]-qq.loc[0,f"01{region.upper()}0000TH50DSXD"], label="Script")
# x
x0 = Binout(filepath=str(sim_dir / c / "binout*")).as_df("nodout", "x_coordinate")[str(node_ids[region])].iloc[0]
ax.plot(
    Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time")*1000,
    -1*(x0 - ds_x_displ_dr[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :]),
    label=region + "Driver",
    ls="-."
)
ax.plot(
    Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time")*1000,
    -1*(x0 - ds_x_displ_pa[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :]),
    label=region + "Passenger",
    ls="-."
)

ax.legend()
ax.grid()

In [ ]:
region = "Head"
c = "Oblique_Left"

fig, ax = plt.subplots()
ax.plot(qq.index, qq[f"01{region.upper()}LOC0TH50DSYD"]-qq.loc[0,f"01{region.upper()}0000TH50DSYD"], label="Script")
# x
y0 = Binout(filepath=str(sim_dir / c / "binout*")).as_df("nodout", "y_coordinate")[str(node_ids[region])].iloc[0]
ax.plot(
    Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time")*1000,
    -1*(y0 - ds_y_displ_front[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :]),
    label=region + "Driver",
    ls="-."
)
ax.plot(
    Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time")*1000,
    -1*(y0 - ds_y_displ_rear[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :]),
    label=region + "Passenger",
    ls="-."
)

ax.legend()
ax.grid()

In [ ]:
region = "Head"
c = "Oblique_Left"

fig, ax = plt.subplots()
ax.plot(qq.index, qq[f"01{region.upper()}LOC0TH50DSZD"]-qq.loc[0,f"01{region.upper()}0000TH50DSZD"], label="Script")
# x
z0 = Binout(filepath=str(sim_dir / c / "binout*")).as_df("nodout", "z_coordinate")[str(node_ids[region])].iloc[0]
ax.plot(
    Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time")*1000,
    -1*(z0 - ds_z_displ_dr[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :]),
    label=region + "Driver",
    ls="-."
)
ax.plot(
    Binout(filepath=str(sim_dir / c / "binout*")).read("nodout", "time")*1000,
    -1*(z0 - ds_z_displ_pa[c][np.nonzero(data[c][0] == node_ids[region])[0][0], :]),
    label=region + "Passenger",
    ls="-."
)

ax.legend()
ax.grid()